# Infer

> Dev notebook

In [1]:
# | default_exp infer

In [2]:
# | hide
%reload_ext autoreload
%reload_ext nb_black
%autoreload 2
from nbdev.showdoc import *
import sys

__root = "../"
sys.path.append(__root)


<IPython.core.display.Javascript object>

In [3]:
# | export
from torch_snippets import *
from clip.dataset import DistilBertTokenizer

<IPython.core.display.Javascript object>

In [4]:
# | export
@torch.no_grad()
def get_image_embeddings(val_dl, model):
    model.eval()
    valid_image_embeddings = []
    for batch in track2(val_dl):
        image_features = model.image_encoder(batch["image"].to(model.config.device))
        image_embeddings = model.image_projection(image_features)
        valid_image_embeddings.append(image_embeddings)
    return torch.cat(valid_image_embeddings)


@torch.no_grad()
def find_matches(model, image_embeddings, query, image_filenames):
    tokenizer = DistilBertTokenizer.from_pretrained(
        model.config.distilbert_text_tokenizer
    )
    encoded_query = tokenizer([query])
    batch = {
        key: torch.tensor(values).to(model.config.device)
        for key, values in encoded_query.items()
    }
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    text_embeddings = model.text_projection(text_features)

    image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)
    text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
    dot_similarity = text_embeddings_n @ image_embeddings_n.T

    values, indices = torch.topk(dot_similarity.squeeze(0), 45)
    matches = [image_filenames[idx] for idx in indices[::5]]
    matches = [f"{model.config.image_path}/{match}" for match in matches]
    subplots(matches, nc=3, figsize=(10, 10))

<IPython.core.display.Javascript object>

In [5]:
# | hide

import nbdev
nbdev.nbdev_export()
import subprocess

subprocess.run(["/home/yyr/anaconda3/envs/mcvp-book/bin/black", __root])


/home/yyr/anaconda3/envs/mcvp-book/lib/python3.8/site-packages/nbdev/export.py:54: UserWarning: Notebook '/mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/nbs/02.00_training.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
Skipping .ipynb files as Jupyter dependencies are not installed.
You can fix this by running ``pip install "black[jupyter]"``
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/core.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/config.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/infer.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/_modidx.py
reformatted /mnt/347832F37832B388/projects/MCVP2e/Chapter-15b/CLIP/clip/dataset.py
reforma

CompletedProcess(args=['/home/yyr/anaconda3/envs/mcvp-book/bin/black', '../'], returncode=0)

<IPython.core.display.Javascript object>